In [ ]:
import argparse
import time
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

class AutoDictionary(dict):
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

name  = "data_"
nodes = ["5653", "55ad", "55e4", "5599", "55dd", "5565", "560b", "5632", "55b3", "5563", "630a"]
#nodes = ["5599"]
ext = '.csv'

names  = ["FSK", "OQPSK", "OFDM"] 
values = ["fsk_rssi", "oqpsk_rssi", "ofdm_rssi"]

packets_per_hour = 3 * 60
total_rx_packets = 0

result = AutoDictionary()

# Iterate over all nodes
for node in nodes:
    print("Parsing node = {} ".format(node))
    
    # Read values from csv
    filename = name + node + ext
    data = pd.read_csv(filename, index_col=0) 
    
    # Create date column based on day and hour
    data['date'] = data['day'] + ' ' + data['hour']
    data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d %H:%M:%S')
    
    # Drop unnecessary columns
    data = data.drop(['day','hour'], axis=1)
    
    # Set date as index
    data = data.set_index(["date"])
    
    # Repeat for all modulations (i.e., FSK, OQPSK, OFDM)
    for v in values:    
        # Count packets received hourly for each retry
        x1 = data[data['pkt_retry'] == 0].groupby(pd.Grouper(freq='H', level='date'))[v].count().reset_index()
        x2 = data[data['pkt_retry'] == 1].groupby(pd.Grouper(freq='H', level='date'))[v].count().reset_index()
        x3 = data[data['pkt_retry'] == 2].groupby(pd.Grouper(freq='H', level='date'))[v].count().reset_index()

        # Accumulate packets received in one hour, sum and limit PDR to 0-100%
        x1[v] = x1[v] + x2[v] + x3[v]
        x1[v] = (100 * x1[v] / packets_per_hour).clip(0, 100)

        # Set index to compute daily averages
        x = x1.set_index(["date"]).groupby(pd.Grouper(freq='D', level='date'))
    
        # Store results
        result[node][v] = x

In [ ]:
nrow = 6
ncol = 2
fig, axs = plt.subplots(nrow, ncol, figsize=(20,20), constrained_layout=True)

for ax, key in zip(enumerate(fig.axes), result.keys()):
    data_fsk   = result[key]["fsk_rssi"]
    data_oqpsk = result[key]["oqpsk_rssi"]
    data_ofdm  = result[key]["ofdm_rssi"]
    
    # Compute FSK
    fsk_mean = data_fsk.mean()['fsk_rssi']
    fsk_max  = data_fsk.max()['fsk_rssi']
    fsk_min  = data_fsk.min()['fsk_rssi']
        
    # Compute OQPSK
    oqpsk_mean = data_oqpsk.mean()['oqpsk_rssi']
    oqpsk_max  = data_oqpsk.max()['oqpsk_rssi']
    oqpsk_min  = data_oqpsk.min()['oqpsk_rssi']
    
    # Compute OFDM
    ofdm_mean = data_ofdm.mean()['ofdm_rssi']
    ofdm_max  = data_ofdm.max()['ofdm_rssi']
    ofdm_min  = data_ofdm.min()['ofdm_rssi']
    
    # Compute dates
    date = fsk_mean.reset_index()['date'].to_numpy()
    
    # Plot FSK
    ax[1].plot(date, fsk_mean, lw=2, label="FSK")
    ax[1].fill_between(date, fsk_max, fsk_min, alpha=0.25)
    
    # Plot OQPSK
    ax[1].plot(date, oqpsk_mean, lw=2, label="OQPSK")
    ax[1].fill_between(date, oqpsk_max, oqpsk_min, alpha=0.25)
    
    # Plot OFDM
    ax[1].plot(date, ofdm_mean, lw=2, label="OFDM")
    ax[1].fill_between(date, ofdm_max, ofdm_min, alpha=0.25)
    
    # Plot gray vertical lines
    ax[1].axvspan(date[19], date[22], alpha=0.66, color='grey')
    ax[1].axvspan(date[88], date[91], alpha=0.66, color='grey')
    
    # Configure plot
    ax[1].legend()
    name = (key[:2] + '-' + key[2:]).upper()
    ax[1].set_title("EUI-64 = {}".format(name))
    ax[1].fmt_xdata = mdates.DateFormatter('%d-%m')
    ax[1].grid()
    ax[1].set_ylim(0, 100)
    ax[1].set_ylabel("PDR (%)")

fig.delaxes(axs[5][1])
fig.suptitle("Packet Delivery Ratio (%)")
plt.savefig("02_pdr_plot.png", dpi=300)